In [1]:
import gradio as gr
import torch
torch_device = "cuda" if torch.cuda.is_available() else "cpu"
from tqdm.auto import tqdm
from torch import autocast
from PIL import Image
import cv2
import numpy as np
import math
import random
import bisect
import operator
import matplotlib.pyplot as plt
import copy
import sys
import os
import shutil
import datetime
import time
np.set_printoptions(threshold=np.inf)

/home/kobayashi/anaconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler

# 1. Load the autoencoder model which will be used to decode the latents into image space. 
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")

# 2. Load the tokenizer and text encoder to tokenize and encode the text. 
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

# 3. The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.4.layer_norm1.bias', 'vision_model.encoder.layers.14.self_attn.k_proj.weight', 'vision_model.encoder.layers.1.self_attn.out_proj.weight', 'vision_model.encoder.layers.23.self_attn.k_proj.bias', 'vision_model.encoder.layers.4.mlp.fc2.bias', 'vision_model.encoder.layers.21.layer_norm1.bias', 'vision_model.pre_layrnorm.bias', 'vision_model.encoder.layers.15.mlp.fc2.weight', 'vision_model.encoder.layers.20.layer_norm2.bias', 'vision_model.encoder.layers.6.self_attn.q_proj.bias', 'vision_model.encoder.layers.20.mlp.fc2.weight', 'vision_model.encoder.layers.13.layer_norm2.weight', 'vision_model.encoder.layers.3.mlp.fc2.bias', 'vision_model.encoder.layers.22.layer_norm1.bias', 'vision_model.encoder.layers.19.mlp.fc2.weight', 'vision_model.encoder.layers.15.self_attn.q_proj.weight', 'vision_model.encoder.layers.9.self_attn.k_proj.bias', 'vision_mode

In [3]:
from diffusers import LMSDiscreteScheduler

scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)

In [4]:

vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device)

In [5]:
def func(latents,text_embeddings):
    
    num_inference_steps = 50            # Number of denoising steps

    guidance_scale = 7.5                # Scale for classifier-free guidance

    #############################################

    uncond_input = tokenizer(
        [""] * batch_size, padding="max_length", return_tensors="pt"
    )
    with torch.no_grad():
      uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]

    ###############################################

    embeddings = torch.cat([uncond_embeddings, text_embeddings])

    #############################################

    scheduler.set_timesteps(num_inference_steps)

    ##########################################

    latents_1 = latents * scheduler.init_noise_sigma

    #############################################



    for t in tqdm(scheduler.timesteps):
      # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
      latent_model_input_a = torch.cat([latents_1] * 2)

      latent_model_input = scheduler.scale_model_input(latent_model_input_a, t)

      # predict the noise residual
      with torch.no_grad():
        noise_pred = unet(latent_model_input, t, encoder_hidden_states=embeddings).sample

      # perform guidance
      noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
      noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

      # compute the previous noisy sample x_t -> x_t-1
      latents_1 = scheduler.step(noise_pred, t, latents_1).prev_sample

    ###############################################
    # scale and decode the image latents with vae
    latents_3 = 1 / 0.18215 * latents_1

    with torch.no_grad():
      image = vae.decode(latents_3).sample


    ##########################################

    image_1 = (image / 2 + 0.5).clamp(0, 1)
    image_2 = image_1.detach().cpu().permute(0, 2, 3, 1).numpy()
    images = (image_2 * 255).round().astype("uint8")
    pil_images = [Image.fromarray(image) for image in images]
    result_image = pil_images[0]
    
    return result_image


In [28]:
#gaの関数
population = 9 #個体数
show_image_x = 3
show_image_y = 3


if show_image_x*show_image_y!=population or population>9:
    print('Error: parameter error', file=sys.stderr)
    sys.exit(1)

initializa_txt_num = 70 #初期化個体においてtxtのベクトルをどれくらい元から変異させるか
batch_size = 1
height = 512                        # default height of Stable Diffusion
width = 512                        # default width of Stable Diffusion
image_mutation_rate = 300
text_mutation_rate = 6000


#seed値を複数個用意
def initialize_gene():
    arr = []
    for i in range(population):
        embeddings = iec_text_embeddings.clone()
        seed_here = random.randrange(10000)
        latents_torch = torch.randn(
          (batch_size, unet.in_channels, height // 8, width // 8),
          generator=torch.manual_seed(seed_here),
        )
        latents = latents_torch.to(torch_device)
        for j in range(initializa_txt_num):
            a = random.randrange(77)
            b = random.randrange(768)
            embeddings[0][a][b] = np.random.randn()
        arr.append([latents,embeddings])
    return arr


def save_image(diffusion_images,epoch,folder):
    os.mkdir(folder+'/epoch'+str(epoch))
    for i in range (len(diffusion_images)):
        diffusion_images[i].save(folder+"/epoch"+str(epoch)+"/"+str(i)+".png")


In [29]:
FOREIGN = 2
mutation_rate = 1.0
def evolve(selected,genes):
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/evolve.txt', 'a') as f:
        print("epoch", iec_live_epoch-1, file=f)
    evolved_genes,selected_genes = [],[]
    for i in range(population):
        if selected[i]:
            selected_genes.append(genes[i])
    delta = population - len(selected_genes)
    x = max(0,delta-FOREIGN)
    if selected_genes:
        for i in range(x):
            # ログ
            with open('./user_experiment_data/'+experiment_id+'/evolve.txt', 'a') as f:
                print("population", i, file=f)
            evolved_genes.extend([mutate(simple_crossover(selected_genes,selected),mutation_rate)])
    else:
        return initialize_gene()
    x = min(FOREIGN,delta)
    for i in range(population):
        if selected[i]:
            # ログ
            with open('./user_experiment_data/'+experiment_id+'/evolve.txt', 'a') as f:
                print("population", i,"mutation", file=f)
    evolved_genes.extend([mutate(selected_genes[i],mutation_rate) for i in range(len(selected_genes))])
    for i in range(x):
        # ログ
        with open('./user_experiment_data/'+experiment_id+'/evolve.txt', 'a') as f:
            print("population", i,"initialize", file=f)
        embeddings = iec_text_embeddings.clone()
        seed_here = random.randrange(10000)
        latents_torch = torch.randn(
          (batch_size, unet.in_channels, height // 8, width // 8),
          generator=torch.manual_seed(seed_here),
        )
        latents = latents_torch.to(torch_device)
        for j in range(initializa_txt_num):
            a = random.randrange(77)
            b = random.randrange(768)
            embeddings[0][a][b] = np.random.randn()
        evolved_genes.append([latents,embeddings])  

                    
    if len(evolved_genes)!=population:
        print('Error: evolve error', file=sys.stderr)
        sys.exit(1)
    return evolved_genes

def simple_crossover(gene_population,selected):
    p1 = random.randint(0,len(gene_population)-1)
    p2 = random.randint(0,len(gene_population)-1)
    gene_cnt = 0
    for i in range(population):
        if selected[i]:
            gene_cnt += 1
            if p1==gene_cnt-1:
                # ログ
                with open('./user_experiment_data/'+experiment_id+'/evolve.txt', 'a') as f:
                    print("crossover1", i, file=f)
            if p2==gene_cnt-1:
                # ログ
                with open('./user_experiment_data/'+experiment_id+'/evolve.txt', 'a') as f:
                    print("crossover2", i, file=f)
    gene1 = gene_population[p1]
    gene2 = gene_population[p2]
    return crossover(gene1,gene2)

def crossover(gene1,gene2):
    new_gene = copy.deepcopy(gene1)
    gene3 = copy.deepcopy(gene2)
    for j in range(4):
        for k in range(64):
            for l in range(64):
                a = random.randrange(2)
                if a==0:
                    new_gene[0][0][j][k][l] = gene3[0][0][j][k][l]
    for j in range(77):
        for k in range(768):
            new_gene[1][0][j][k] = gene3[1][0][j][k]
    return new_gene

def mutate(individual,mutation_rate = 1.0):
    new_gene = copy.deepcopy(individual)
    for j in range(4):
        for k in range(64):
            for l in range(64):
                a = random.randrange(image_mutation_rate)
                if a==0:
                    value = individual[0][0][j][k][l]+np.random.randn()
                    if value >= 5.00:
                        value = 4.99
                    elif value <=-5.00:
                        value = -4.99
                    new_gene[0][0][j][k][l] = value
    for j in range(77):
        for k in range(768):
            a = random.randrange(text_mutation_rate)
            if a==0:
                value = individual[1][0][j][k]+np.random.randn()
                if value >= 5.00:
                    value = 4.99
                elif value <=-5.00:
                    value = -4.99
                new_gene[1][0][j][k] = value
    return new_gene

In [43]:
import random
import PIL
from PIL import Image,ImageTk
iec_live_epoch=0
conv_live_epoch=0
iec_limit_epoch = 10
conv_limit_epoch = 10
practice_limit_epoch = 3
conv_practice_epoch = 0
iec_practice_epoch = 0
iec_selected_image_id = 0
conv_selected_image_id = 0
iec_selected = []
for i in range(population):
    iec_selected.append(0)


#本番、２回目以降のiecで画像を更新
def generate_iec_images(*iec_selected_image):
    global iec_live_epoch,iec_selected
    outputs = []
    #上限のときは、さっき表示されていた画像が表示される
    if iec_live_epoch==iec_limit_epoch:
        outputs.append("上限に到達しました。画像を一枚選んで終わるボタンを押してください")
        for i in range(population):
            img = Image.open(iec_image_save_path+'/epoch'+str(iec_live_epoch)+'/' + str(i) + '.png').resize((256,256),Image.ANTIALIAS)
            outputs.append(img)
            outputs.append(False)
        return outputs
    for i in range(population):
        if iec_selected_image[i]:
            iec_selected[i]=1
        else:
            iec_selected[i]=0
    iec_live_epoch += 1
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/iec_select.txt', 'a') as f:
        print(time.time(), "epoch"+str(iec_live_epoch-1),
              iec_selected, file=f)
    #進化
    global genes
    genes = evolve(iec_selected,genes)

    #画像生成
    iec_images = []
    outputs.append("試行回数: "+str(iec_live_epoch)+"  残り回数: "+str(iec_limit_epoch-iec_live_epoch))
    with autocast("cuda"):
        for i in range(population):
            iec_image = func(genes[i][0],genes[i][1])
            iec_images.append(iec_image)
            outputs.append(iec_image.resize((256,256),Image.ANTIALIAS))
            outputs.append(False)
    save_image(iec_images,iec_live_epoch,iec_image_save_path)
    for i in range(population):
        with open('./user_experiment_data/'+experiment_id+"/iec/epoch"+str(iec_live_epoch)+"/"+str(i)+"text.txt", 'a') as f:
            x_tensor = genes[i][1]
            device = x_tensor.device
            x_numpy = x_tensor.to('cpu').detach().numpy().copy()
            x_tensor = x_tensor.to(device)
            print(x_numpy,file=f)
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/iec_time_log.txt', 'a') as f:
        print(time.time(), "finished_generaging_image_epoch"+str(iec_live_epoch),file=f)
    return outputs
#本番、２回目以降のconvで画像を更新
def generate_conv_images(input_prompt):
    global conv_live_epoch
    #上限のときは、さっき表示されていた画像が表示される
    if conv_live_epoch==conv_limit_epoch:
        conv_images = []
        conv_images.append("上限に到達しました。画像を一枚選んで終わるボタンを押してください")
        for i in range(population):
            img = Image.open(conv_image_save_path+'/epoch'+str(conv_live_epoch)+'/' + str(i) + '.png').resize((256,256),Image.ANTIALIAS)
            conv_images.append(img)
        return conv_images
    conv_live_epoch += 1
    
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/conv_prompt.txt', 'a') as f:
        print("epoch"+str(conv_live_epoch),
              input_prompt, file=f)
    text_input = tokenizer([input_prompt], padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
    with torch.no_grad():
        conv_text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
    conv_images = []
    conv_images.append("試行回数: "+str(conv_live_epoch)+"  残り回数: "+str(conv_limit_epoch-conv_live_epoch))
    for i in range(population):
        seed_here = random.randrange(10000)
        latents_torch = torch.randn(
          (batch_size, unet.in_channels, height // 8, width // 8),
          generator=torch.manual_seed(seed_here),
        )
        latents = latents_torch.to(torch_device)
        with autocast("cuda"):
            conv_image = func(latents,conv_text_embeddings)
        conv_images.append(conv_image)
    save_image(conv_images[1:],conv_live_epoch,conv_image_save_path)
    for i in range(1,population+1):
        conv_images[i]=conv_images[i].resize((256,256),Image.ANTIALIAS)
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/conv_time_log.txt', 'a') as f:
        print(time.time(), "finished_generaging_image_epoch"+str(conv_live_epoch),file=f)
    return conv_images
# 本番、一回目のiec画像生成
def generate_iec_initial_images(input_prompt):
    iec_images = []
    outputs = []
    global iec_live_epoch
    iec_live_epoch += 1
    outputs.append("試行回数: "+str(iec_live_epoch)+"  残り回数: "+str(iec_limit_epoch-iec_live_epoch))
    text_input = tokenizer([input_prompt], padding="max_length", 
    max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
    global iec_text_embeddings
    with torch.no_grad():
        iec_text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
    global genes
    genes = initialize_gene()
    with autocast("cuda"):
        for i in range(population):
            iec_image = func(genes[i][0],genes[i][1])
            iec_images.append(iec_image)
            outputs.append(iec_image.resize((256,256),Image.ANTIALIAS))
            outputs.append(False)
    save_image(iec_images,iec_live_epoch,iec_image_save_path)
    for i in range(population):
        with open('./user_experiment_data/'+experiment_id+"/iec/epoch"+str(iec_live_epoch)+"/"+str(i)+"text.txt", 'a') as f:
            x_tensor = genes[i][1]
            device = x_tensor.device
            x_numpy = x_tensor.to('cpu').detach().numpy().copy()
            x_tensor = x_tensor.to(device)
            print(x_numpy,file=f)
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/iec_time_log.txt', 'a') as f:
        print(time.time(), "finished_generaging_image_epoch"+str(iec_live_epoch),file=f)
    return outputs
# 本番、一回目のconv画像生成
def generate_conv_initial_images(input_prompt):
    conv_images = []
    global conv_live_epoch,conv_text_embeddings

    conv_live_epoch += 1
    conv_images.append("試行回数: "+str(conv_live_epoch)+"  残り回数: "+str(conv_limit_epoch-conv_live_epoch))
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/conv_prompt.txt', 'a') as f:
        print("epoch"+str(conv_live_epoch),
              input_prompt, file=f)
    text_input = tokenizer([input_prompt], padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
    with torch.no_grad():
        conv_text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
    
    for i in range(population):
        seed_here = random.randrange(10000)
        latents_torch = torch.randn(
          (batch_size, unet.in_channels, height // 8, width // 8),
          generator=torch.manual_seed(seed_here),
        )
        latents = latents_torch.to(torch_device)
        with autocast("cuda"):
            conv_image = func(latents,conv_text_embeddings)
        conv_images.append(conv_image)
    save_image(conv_images[1:],conv_live_epoch,conv_image_save_path)
    for i in range(1,population+1):
        conv_images[i]=conv_images[i].resize((256,256),Image.ANTIALIAS)
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/conv_time_log.txt', 'a') as f:
        print(time.time(), "finished_generaging_image_epoch"+str(conv_live_epoch),file=f)
    return conv_images
#練習用の画像を返す
def generate_iec_fake_images():
    global iec_practice_epoch
    #上限のときは、さっき表示されていた画像が表示される
    if iec_practice_epoch==practice_limit_epoch:
        iec_images = []
        iec_images.append("上限に到達しました。画像を一枚選んで終わるボタンを押してください")
        for i in range(population):
            img = Image.open("sample_image/iec/epoch"+str(iec_practice_epoch)+"/"+str(i)+".png").resize((256,256),Image.ANTIALIAS)
            iec_images.append(img)
            iec_images.append(False)
        return iec_images
    iec_practice_epoch+=1
    images = []
    images.append("試行回数: "+str(iec_practice_epoch)+"  残り回数: "
                  +str(practice_limit_epoch-iec_practice_epoch))
    for i in range(population):
        image = Image.open("sample_image/iec/epoch"+str(iec_practice_epoch)+"/"+str(i)+".png").resize((256,256),Image.ANTIALIAS)
        images.append(image)
        images.append(False)
    return images
#練習用の画像を返す
def generate_conv_fake_images():
    global conv_practice_epoch
    #上限のときは、さっき表示されていた画像が表示される
    if conv_practice_epoch==practice_limit_epoch:
        conv_images = []
        conv_images.append("上限に到達しました。画像を一枚選んで終わるボタンを押してください")
        for i in range(population):
            img = Image.open("sample_image/iec/epoch"+str(conv_practice_epoch)+"/"+str(i)+".png").resize((256,256),Image.ANTIALIAS)
            conv_images.append(img)
        return conv_images
    conv_practice_epoch+=1
    images = []
    images.append("試行回数: "+str(conv_practice_epoch)+"  残り回数: "
                  +str(practice_limit_epoch-conv_practice_epoch))
    for i in range(population):
        image = Image.open("sample_image/iec/epoch"+str(conv_practice_epoch)+"/"+str(i)+".png").resize((256,256),Image.ANTIALIAS)
        images.append(image)
    return images
def decide_iec_image(index):
    global iec_selected_image_id
    iec_selected_image_id = index
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/iec_time_log.txt', 'a') as f:
        print(time.time(), "finish_button_clicked_epoch"+str(iec_live_epoch), file=f)
    return "お疲れ様です"
def decide_conv_image(index):
    global conv_selected_image_id
    conv_selected_image_id = index
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/conv_time_log.txt', 'a') as f:
        print(time.time(), "finish_button_clicked_epoch"+str(conv_live_epoch), file=f)
    return "お疲れ様です"
    
#結果画像の表示
def show_result_image():
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/result.txt', 'a') as f:
        print("iec epoch",iec_live_epoch,"image", iec_selected_image_id,file=f)
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/result.txt', 'a') as f:
        print("conv epoch",conv_live_epoch,"image", conv_selected_image_id,file=f)
    iec_image = Image.open(iec_image_save_path
                           +'/epoch'+str(iec_live_epoch)+'/' + str(iec_selected_image_id) + '.png')
    conv_image = Image.open(conv_image_save_path+'/epoch'+
                            str(conv_live_epoch)+'/' + str(conv_selected_image_id) + '.png')
    return [iec_image,conv_image]
def reading_text_file(file_name):
    file = open(file_name, 'r')
    text_data = file.read()
    return text_data

#convの画像生成
def conv_update_click(text_prompt):
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/conv_time_log.txt', 'a') as f:
        print(time.time(), "update_button_clicked_epoch"+str(conv_live_epoch), file=f)
    if conv_live_epoch==0:
        return generate_conv_initial_images(text_prompt)
    else:
        return generate_conv_images(text_prompt)
#iecの画像生成
def iec_update_click(input_prompt,*iec_selected_image):
    # ログ
    with open('./user_experiment_data/'+experiment_id+'/iec_time_log.txt', 'a') as f:
        print(time.time(), "update_button_clicked_epoch"+str(iec_live_epoch), file=f)
    if iec_live_epoch==0:
        return generate_iec_initial_images(input_prompt)
    else:
        return generate_iec_images(*iec_selected_image)
def fake_end():
    return "お疲れ様でした"
def show_iec_current_images():
    iec_images = []
    if iec_live_epoch==0:
        return generate_iec_fake_images()
    iec_images.append("試行回数: "+str(iec_live_epoch)+"  残り回数: "+str(iec_limit_epoch-iec_live_epoch))
    for i in range(population):
        img = Image.open(iec_image_save_path+'/epoch'+str(iec_live_epoch)+'/' + str(i) + '.png').resize((256,256),Image.ANTIALIAS)
        iec_images.append(img)
        iec_images.append(False)
    return iec_images
def show_conv_current_images():
    conv_images = []
    if conv_live_epoch==0:
        return generate_conv_fake_images()
    conv_images.append("試行回数: "+str(conv_live_epoch)+"  残り回数: "+str(conv_limit_epoch-conv_live_epoch))
    for i in range(population):
        img = Image.open(conv_image_save_path+'/epoch'+str(conv_live_epoch)+'/' + str(i) + '.png').resize((256,256),Image.ANTIALIAS)
        conv_images.append(img)
    return conv_images
with gr.Blocks() as demo:
    with gr.Tab("練習 1"):
        output = []
        population_check = []
        with gr.Row():
            with gr.Column():
                gr.Markdown("### お題: "+reading_text_file('src/practice_topic.txt'))
                gr.Markdown(reading_text_file('src/iec_howto.txt'))
                input_prompt = gr.Textbox(label="文章を入力してください",
                                          lines=5,value = reading_text_file('src/practice_input_text.txt'))
                guidline = gr.Textbox("試行回数: "+str(iec_practice_epoch)+"  残り回数: "
                                         +str(practice_limit_epoch-iec_practice_epoch),show_label=False)
                output.append(guidline)
                continue_btn = gr.Button("画像を更新")
                iec_select_dropdown=gr.Dropdown(list(range(population)),label="気に入った画像"),
                end_btn = gr.Button("終わる")
            for i in range(show_image_x):    
                with gr.Column():
                    for j in range(show_image_y):
                        with gr.Row():
                            output.append(gr.Image(label=str(i*show_image_y+j)).style(height=200,width=200))
                        pop_check = gr.Checkbox(label="👍",interactive=True)
                        population_check.append(pop_check)
                        output.append(pop_check)
        continue_btn.click(fn=generate_iec_fake_images, inputs=None, outputs=output)
        end_btn.click(fn=fake_end,inputs=None,outputs=guidline)
    with gr.Tab("練習 2"):
        output = []
        population_check = []
        with gr.Row():
            with gr.Column():
                gr.Markdown("### お題: "+reading_text_file('src/practice_topic.txt'))
                gr.Markdown(reading_text_file('src/conv_howto.txt'))
                input_prompt = gr.Textbox(label="文章を入力してください",
                                        lines=5,value = reading_text_file('src/practice_input_text.txt'),
                                         interactive=True)
                guidline = gr.Textbox("試行回数: "+str(conv_practice_epoch)+"  残り回数: "
                                         +str(practice_limit_epoch-conv_practice_epoch),show_label=False)
                output.append(guidline)
                continue_btn = gr.Button("画像を更新")
                iec_select_dropdown=gr.Dropdown(list(range(population)),label="気に入った画像"),
                end_btn = gr.Button("終わる")
            for i in range(show_image_x):    
                with gr.Column():
                    for j in range(show_image_y):
                        with gr.Row():
                            output.append(gr.Image(label=str(i*show_image_y+j)).style(height=200,width=200))
        continue_btn.click(fn=generate_conv_fake_images, inputs=None, outputs=output)
        end_btn.click(fn=fake_end,inputs=None,outputs=guidline)
    with gr.Tab("実験 1"):
        output = []
        continue_input = []
        with gr.Row():
            with gr.Column():
                gr.Markdown("### お題: "+reading_text_file('src/topic1.txt'))
                gr.Markdown(reading_text_file('src/iec_howto.txt'))
                continue_input.append(gr.Textbox(label="文章を入力してください",
                lines=5,value = reading_text_file('src/prompt1.txt'),interactive=False))
                guidline = gr.Textbox("試行回数: "+str(iec_live_epoch)
                                         +"  残り回数: "+str(iec_limit_epoch-iec_live_epoch),show_label=False)
                output.append(guidline)
                continue_btn = gr.Button("画像を更新")
                iec_select_dropdown=gr.Dropdown(list(range(population)),label="気に入った画像")
                end_btn = gr.Button("終わる")
                reset_btn = gr.Button("リセット")
            for i in range(show_image_x):    
                with gr.Column():
                    for j in range(show_image_y):
                        with gr.Row():
                            output.append(gr.Image(label=str(i*show_image_y+j)).style(height=200,width=200))
                        pop_check = gr.Checkbox(label="👍",interactive=True)
                        continue_input.append(pop_check)
                        output.append(pop_check)
        continue_btn.click(fn=iec_update_click, inputs=continue_input, outputs=output)
        end_btn.click(fn=decide_iec_image,inputs=iec_select_dropdown,outputs=guidline)
        reset_btn.click(fn=show_iec_current_images,inputs=None,outputs=output)
    with gr.Tab("実験 2"):
        output = []
        population_check = []
        with gr.Row():
            with gr.Column():
                gr.Markdown("### お題: "+reading_text_file('src/topic2.txt'))
                gr.Markdown(reading_text_file('src/conv_howto.txt'))
                input_prompt = gr.Textbox(label="文章を入力してください",
                lines=5,value = reading_text_file('src/prompt2.txt'),interactive=True)
                guidline = gr.Textbox("試行回数: "+str(conv_live_epoch)+"  残り回数: "
                                         +str(conv_limit_epoch-conv_live_epoch),show_label=False)
                output.append(guidline)
                continue_btn = gr.Button("画像を更新")
                conv_select_dropdown=gr.Dropdown(list(range(population)),label="気に入った画像")
                end_btn = gr.Button("終わる")
                reset_btn = gr.Button("リセット")
            for i in range(show_image_x):    
                with gr.Column():
                    for j in range(show_image_y):
                        with gr.Row():
                            output.append(gr.Image(label=str(i*show_image_y+j)).style(height=200,width=200))
        continue_btn.click(fn=conv_update_click, inputs=input_prompt, outputs=output)
        end_btn.click(fn=decide_conv_image,inputs=conv_select_dropdown,outputs=guidline)
        reset_btn.click(fn=show_conv_current_images,inputs=None,outputs=output)
    with gr.Tab("結果"):
        gr.Markdown("experiment_id: "+str(experiment_id))
        selected_images = []
        with gr.Row():
            with gr.Column():
                gr.Markdown("### お題: "+reading_text_file('src/topic1.txt'))
                selected_images.append(gr.Image(label="実験1").style(height=300,width=300))
            with gr.Column():
                gr.Markdown("### お題: "+reading_text_file('src/topic2.txt'))
                selected_images.append(gr.Image(label="実験2").style(height=300,width=300))
        show_btn = gr.Button("結果を表示する")
        show_btn.click(fn=show_result_image,inputs=None,outputs=selected_images)
if __name__ == "__main__":
    #被験者のidを振る
    experiment_time_delta = datetime.timedelta(hours=9)
    experiment_JST = datetime.timezone(experiment_time_delta, 'JST')
    global experiment_id,iec_image_save_path,conv_image_save_path
    experiment_now = datetime.datetime.now(experiment_JST)
    experiment_id = experiment_now.strftime('%Y%m%d%H%M%S')
    iec_image_save_path='./user_experiment_data/'+experiment_id+'/iec'
    conv_image_save_path='./user_experiment_data/'+experiment_id+'/conv'
    #ディレクトリ初期化
    os.mkdir('./user_experiment_data/'+experiment_id)
    os.mkdir(iec_image_save_path)
    os.mkdir(conv_image_save_path)
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7894
Running on public URL: https://e1f0a3fe-0ed1-43a4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


100%|██████████| 50/50 [00:05<00:00,  8.41it/s]
/tmp/ipykernel_55224/414557727.py:122: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  outputs.append(iec_image.resize((256,256),Image.ANTIALIAS))
100%|██████████| 50/50 [00:05<00:00,  9.08it/s]
/tmp/ipykernel_55224/414557727.py:265: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = Image.open(iec_image_save_path+'/epoch'+str(iec_live_epoch)+'/' + str(i) + '.png').resize((256,256),Image.ANTIALIAS)
100%|██████████| 50/50 [00:05<00:00,  9.09it/s]
/tmp/ipykernel_55224/414557727.py:51: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  outputs.append(iec_image.resize((256,256),Image.ANTIALIAS))
100%|██████████| 50/50 [00:05<00:00,  8.37it/s]
/tmp/ipykernel_55224/414557727.py:26: DeprecationWarning: ANTIALIAS is dep

In [31]:
#debug用
def show_iec_current_images():
    iec_images = []
    iec_images.append("試行回数: "+str(iec_live_epoch)+"  残り回数: "+str(iec_limit_epoch-iec_live_epoch))
    for i in range(population):
        img = Image.open(iec_image_save_path+'/epoch'+str(iec_live_epoch)+'/' + str(i) + '.png').resize((256,256),Image.ANTIALIAS)
        iec_images.append(img)
    return iec_images
def show_conv_current_images():
    conv_images = []
    conv_images.append("試行回数: "+str(conv_live_epoch)+"  残り回数: "+str(conv_limit_epoch-conv_live_epoch))
    for i in range(population):
        img = Image.open(conv_image_save_path+'/epoch'+str(conv_live_epoch)+'/' + str(i) + '.png').resize((256,256),Image.ANTIALIAS)
        conv_images.append(img)
    return conv_images
with gr.Blocks() as demo:
    with gr.Tab("実験 1"):
        output = []
        continue_input = []
        with gr.Row():
            with gr.Column():
                gr.Markdown(reading_text_file('src/iec_howto.txt'))
                continue_input.append(gr.Textbox(label="文章を入力してください"))
                guidline = gr.Textbox("試行回数: "+str(iec_live_epoch)
                                         +"  残り回数: "+str(iec_limit_epoch-iec_live_epoch),label=None)
                output.append(guidline)
                current_btn = gr.Button("現在の画像を表示")
                continue_btn = gr.Button("続ける")
                iec_select_dropdown=gr.Dropdown(list(range(population)),label="気に入った画像")
                end_btn = gr.Button("終わる")
            for i in range(show_image_x):    
                with gr.Column():
                    for j in range(show_image_y):
                        with gr.Row():
                            output.append(gr.Image(label=str(i*show_image_y+j)).style(height=200,width=200))
                        continue_input.append(gr.Checkbox(label="👍"))
        current_btn.click(fn=show_iec_current_images,inputs=None,outputs=output)
        continue_btn.click(fn=iec_update_click, inputs=continue_input, outputs=output)
        end_btn.click(fn=decide_iec_image,inputs=iec_select_dropdown,outputs=guidline)
    with gr.Tab("実験 2"):
        output = []
        population_check = []
        with gr.Row():
            with gr.Column():
                gr.Markdown(reading_text_file('src/conv_howto.txt'))
                input_prompt = gr.Textbox(label="文章を入力してください")
                guidline = gr.Textbox("試行回数: "+str(conv_live_epoch)+"  残り回数: "
                                         +str(conv_limit_epoch-conv_live_epoch),label=None)
                output.append(guidline)
                current_btn = gr.Button("現在の画像を表示")
                continue_btn = gr.Button("続ける")
                conv_select_dropdown=gr.Dropdown(list(range(population)),label="気に入った画像")
                end_btn = gr.Button("終わる")
            for i in range(show_image_x):    
                with gr.Column():
                    for j in range(show_image_y):
                        with gr.Row():
                            output.append(gr.Image(label=str(i*show_image_y+j)).style(height=200,width=200))
        current_btn.click(fn=show_conv_current_images,inputs=None,outputs=output)
        continue_btn.click(fn=conv_update_click, inputs=input_prompt, outputs=output)
        end_btn.click(fn=decide_conv_image,inputs=conv_select_dropdown,outputs=guidline)
    with gr.Tab("結果"):
        show_btn = gr.Button("結果を表示する")
        with gr.Row():
            selected_images = [gr.Image(label="実験1").style(height=200,width=200),gr.Image(label="実験2").style(height=200,width=200)]
        show_btn.click(fn=show_result_image,inputs=None,outputs=selected_images)
    demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7880
Running on public URL: https://21ec9abf-8407-41ac.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


100%|██████████| 50/50 [00:05<00:00,  8.56it/s]
/tmp/ipykernel_55224/3693576669.py:169: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  conv_images[i]=conv_images[i].resize((256,256),Image.ANTIALIAS)
100%|██████████| 50/50 [00:05<00:00,  8.47it/s]
/tmp/ipykernel_55224/3693576669.py:137: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  iec_images[i]=iec_images[i].resize((256,256),Image.ANTIALIAS)
100%|██████████| 50/50 [00:05<00:00,  8.53it/s]
/tmp/ipykernel_55224/3693576669.py:101: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  conv_images[i]=conv_images[i].resize((256,256),Image.ANTIALIAS)


In [20]:
image = Image.open("sample_image/iec/epoch0/0.png")
r = [image,image]
p = [x.resize((256,256),Image.ANTIALIAS) in r]
# r.resize((256,256),Image.ANTIALIAS)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 3>                                                                                │
│                                                                                                  │
│   1 image = Image.open("sample_image/iec/epoch0/0.png")                                          │
│   2 r = [image,image]                                                                            │
│ ❱ 3 p = [x.resize((256,256),Image.ANTIALIAS) in r]                                               │
│   4 # r.resize((256,256),Image.ANTIALIAS)                                                        │
│   5                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'x' is not defined